# PySpark and SparkR Arrow optimization examples

First, enable R cell magic as below.

In [1]:
import rpy2.rinterface
%load_ext rpy2.ipython

After that, locate SparkR library directroy for SparkR.

In [2]:
%%R
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

R[write to console]: 
Attaching package: ‘SparkR’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window


R[write to console]: The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union




Initializes a Spark session for PySaprk as well.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## R DataFrame <> Spark DataFrame with Arrow optimization

In [4]:
%%R
sparkR.session(master = "local[*]", sparkConfig = list(spark.sql.execution.arrow.sparkr.enabled = "true"))

R[write to console]: Spark package found in SPARK_HOME: /home/jovyan/spark



Launching java with spark-submit command /home/jovyan/spark/bin/spark-submit   sparkr-shell /tmp/Rtmp8Fl65g/backend_port29763431a86 
Java ref type org.apache.spark.sql.SparkSession id 1 


In [5]:
%%R
collect(createDataFrame(iris))

    Sepal_Length Sepal_Width Petal_Length Petal_Width    Species
1            5.1         3.5          1.4         0.2     setosa
2            4.9         3.0          1.4         0.2     setosa
3            4.7         3.2          1.3         0.2     setosa
4            4.6         3.1          1.5         0.2     setosa
5            5.0         3.6          1.4         0.2     setosa
6            5.4         3.9          1.7         0.4     setosa
7            4.6         3.4          1.4         0.3     setosa
8            5.0         3.4          1.5         0.2     setosa
9            4.4         2.9          1.4         0.2     setosa
10           4.9         3.1          1.5         0.1     setosa
11           5.4         3.7          1.5         0.2     setosa
12           4.8         3.4          1.6         0.2     setosa
13           4.8         3.0          1.4         0.1     setosa
14           4.3         3.0          1.1         0.1     setosa
15           5.8         

126          7.2         3.2          6.0         1.8  virginica
127          6.2         2.8          4.8         1.8  virginica
128          6.1         3.0          4.9         1.8  virginica
129          6.4         2.8          5.6         2.1  virginica
130          7.2         3.0          5.8         1.6  virginica
131          7.4         2.8          6.1         1.9  virginica
132          7.9         3.8          6.4         2.0  virginica
133          6.4         2.8          5.6         2.2  virginica
134          6.3         2.8          5.1         1.5  virginica
135          6.1         2.6          5.6         1.4  virginica
136          7.7         3.0          6.1         2.3  virginica
137          6.3         3.4          5.6         2.4  virginica
138          6.4         3.1          5.5         1.8  virginica
139          6.0         3.0          4.8         1.8  virginica
140          6.9         3.1          5.4         2.1  virginica
141          6.7         

## `dapply` and `gapply` with Arrow optimization

In [6]:
%%R
sparkR.session(master = "local[*]", sparkConfig = list(spark.sql.execution.arrow.sparkr.enabled = "true"))

Java ref type org.apache.spark.sql.SparkSession id 1 


In [7]:
%%R
df <- createDataFrame(mtcars)
collect(gapply(df,
               "gear",
               function(key, group) {
                 data.frame(gear = key[[1]], disp = mean(group$disp) > group$disp)
               },
               structType("gear double, disp boolean")))

   gear  disp
1     4 FALSE
2     4 FALSE
3     4  TRUE
4     4 FALSE
5     4 FALSE
6     4 FALSE
7     4 FALSE
8     4  TRUE
9     4  TRUE
10    4  TRUE
11    4  TRUE
12    4  TRUE
13    3  TRUE
14    3 FALSE
15    3  TRUE
16    3 FALSE
17    3  TRUE
18    3  TRUE
19    3  TRUE
20    3 FALSE
21    3 FALSE
22    3 FALSE
23    3  TRUE
24    3  TRUE
25    3  TRUE
26    3 FALSE
27    3 FALSE
28    5  TRUE
29    5  TRUE
30    5 FALSE
31    5  TRUE
32    5 FALSE


In [8]:
%%R
df <- createDataFrame(mtcars)
collect(dapply(df, function(rdf) { data.frame(rdf$gear + 1) }, structType("gear double")))

   gear
1     5
2     5
3     5
4     4
5     4
6     4
7     4
8     5
9     5
10    5
11    5
12    4
13    4
14    4
15    4
16    4
17    4
18    5
19    5
20    5
21    4
22    4
23    4
24    4
25    4
26    5
27    6
28    6
29    6
30    6
31    6
32    5


## Pandas DataFrame <> Spark DataFrame with Arrow optimization

In [9]:
import pandas as pd

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.createDataFrame(pd.DataFrame({'a': range(10)})).toPandas()

,a
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
